# Example - URBANopt Project with REopt Gem 

In [21]:
require 'openstudio/extension'
require 'openstudio/extension/rake_task'
require 'urbanopt/scenario'
require 'urbanopt/geojson'
require 'urbanopt/reopt_scenario'
require 'urbanopt/reopt'
require 'daru'

false

In [22]:
DEVELOPER_NREL_KEY = '2we3uMoY5IDi8HUMIUYuinl734RNmC5xF7iOXqbg'

<main>: warning: already initialized constant DEVELOPER_NREL_KEY


"2we3uMoY5IDi8HUMIUYuinl734RNmC5xF7iOXqbg"

## Setup

In [23]:
def baseline_scenario
  root_dir = File.absolute_path(File.dirname(__FILE__))
  name = 'Baseline Scenario'
  run_dir = File.join(File.dirname(__FILE__), 'run/baseline_scenario/')
  feature_file_path = File.join(File.dirname(__FILE__), 'example_project.json')
  csv_file = File.join(File.dirname(__FILE__), 'baseline_scenario.csv')
  mapper_files_dir = File.join(File.dirname(__FILE__), '../mappers/')
  reopt_files_dir = File.join(File.dirname(__FILE__), '../reopt/')
  scenario_reopt_assumptions_file_name = 'base_assumptions.json'
  num_header_rows = 1

  feature_file = URBANopt::GeoJSON::GeoFile.from_file(feature_file_path)
  scenario = URBANopt::Scenario::REoptScenarioCSV.new(name, root_dir, run_dir, feature_file, mapper_files_dir, csv_file, num_header_rows, reopt_files_dir, scenario_reopt_assumptions_file_name)
  return scenario
end


:baseline_scenario

In [24]:
csv_file = File.join(File.dirname(__FILE__), 'baseline_scenario.csv')
Daru::DataFrame.from_csv csv_file

#<Daru::DataFrame(3x4)>
            Feature Id Feature Na Mapper Cla REopt Assu
          0          1 Mixed_use  URBANopt:: base_assum
          1          2 Restaurant URBANopt:: base_assum
          2          3 Residentia URBANopt:: base_assum

In [25]:
feature_reports_hash = nil
File.open('../reopt/base_assumptions.json', 'r') do |file|
  feature_reports_hash = JSON.parse(file.read)
end

p "PV", feature_reports_hash["Scenario"]['Site']['PV']
p "Storage", feature_reports_hash["Scenario"]['Site']['Storage']
p "ElectricTariff", feature_reports_hash["Scenario"]['Site']['ElectricTariff']
p "Financial", feature_reports_hash["Scenario"]['Site']['Financial']


"PV"
{"existing_kw"=>0, "min_kw"=>0, "max_kw"=>1000000000.0, "installed_cost_us_dollars_per_kw"=>1600, "om_cost_us_dollars_per_kw"=>16, "macrs_option_years"=>5, "macrs_bonus_pct"=>1, "macrs_itc_reduction"=>0.5, "federal_itc_pct"=>0.26, "state_ibi_pct"=>0, "state_ibi_max_us_dollars"=>10000000000.0, "utility_ibi_pct"=>0, "utility_ibi_max_us_dollars"=>10000000000.0, "federal_rebate_us_dollars_per_kw"=>0, "state_rebate_us_dollars_per_kw"=>0, "state_rebate_max_us_dollars"=>10000000000.0, "utility_rebate_us_dollars_per_kw"=>0, "utility_rebate_max_us_dollars"=>10000000000.0, "pbi_us_dollars_per_kwh"=>0, "pbi_max_us_dollars"=>1000000000.0, "pbi_years"=>1, "pbi_system_max_kw"=>1000000000.0, "degradation_pct"=>0.005, "azimuth"=>180, "losses"=>0.14, "array_type"=>1, "module_type"=>0, "gcr"=>0.4, "dc_ac_ratio"=>1.2, "inv_eff"=>0.96, "radius"=>0, "tilt"=>0.537}
"Storage"
{"min_kw"=>0, "max_kw"=>1000000, "min_kwh"=>0, "max_kwh"=>1000000, "internal_efficiency_pct"=>0.975, "inverter_efficiency_pct"=>0

["Financial", {"om_cost_escalation_pct"=>0.025, "escalation_pct"=>0.023, "offtaker_tax_pct"=>0.26, "offtaker_discount_pct"=>0.083, "analysis_years"=>25, "value_of_lost_load_us_dollars_per_kwh"=>100, "microgrid_upgrade_cost_pct"=>0.3}]

## Post Processing

In [27]:
default_post_processor = URBANopt::Scenario::ScenarioDefaultPostProcessor.new(baseline_scenario)
scenario_report = default_post_processor.run
scenario_base = default_post_processor.scenario_base
scenario_report.save

true

In [28]:
reopt_post_processor = URBANopt::REopt::REoptPostProcessor.new(scenario_report,scenario_base.scenario_reopt_assumptions_file, scenario_base.reopt_feature_assumptions, DEVELOPER_NREL_KEY) 
true

true

In [29]:
p "Scenario Level Assumptions", reopt_post_processor.scenario_reopt_default_assumptions_hash
p
p "Feature Level Assumptions", reopt_post_processor.feature_reports_reopt_default_assumption_hashes

"Scenario Level Assumptions"
{:Scenario=>{:Site=>{:Financial=>{:om_cost_escalation_pct=>0.025, :escalation_pct=>0.023, :offtaker_tax_pct=>0.26, :offtaker_discount_pct=>0.083, :analysis_years=>25, :value_of_lost_load_us_dollars_per_kwh=>100, :microgrid_upgrade_cost_pct=>0.3}, :LoadProfile=>{:year=>2018, :loads_kw_is_net=>true, :critical_loads_kw_is_net=>false, :critical_load_pct=>0.5, :outage_is_major_event=>true}, :ElectricTariff=>{:add_blended_rates_to_urdb_rate=>false, :net_metering_limit_kw=>0, :interconnection_limit_kw=>100000000.0, :urdb_label=>"5e162e2a5457a3d50873e3af"}, :Wind=>{:min_kw=>0, :max_kw=>0, :installed_cost_us_dollars_per_kw=>3013, :om_cost_us_dollars_per_kw=>40, :macrs_option_years=>5, :macrs_bonus_pct=>1, :macrs_itc_reduction=>0.5, :federal_itc_pct=>0.26, :state_ibi_pct=>0, :state_ibi_max_us_dollars=>10000000000.0, :utility_ibi_pct=>0, :utility_ibi_max_us_dollars=>10000000000.0, :federal_rebate_us_dollars_per_kw=>0, :state_rebate_us_dollars_per_kw=>0, :state_rebate_

[{:Scenario=>{:Site=>{:Financial=>{:om_cost_escalation_pct=>0.025, :escalation_pct=>0.023, :offtaker_tax_pct=>0.26, :offtaker_discount_pct=>0.083, :analysis_years=>25, :value_of_lost_load_us_dollars_per_kwh=>100, :microgrid_upgrade_cost_pct=>0.3}, :LoadProfile=>{:year=>2018, :loads_kw_is_net=>true, :critical_loads_kw_is_net=>false, :critical_load_pct=>0.5, :outage_is_major_event=>true}, :ElectricTariff=>{:add_blended_rates_to_urdb_rate=>false, :net_metering_limit_kw=>0, :interconnection_limit_kw=>100000000.0, :urdb_label=>"5e162e2a5457a3d50873e3af"}, :Wind=>{:min_kw=>0, :max_kw=>0, :installed_cost_us_dollars_per_kw=>3013, :om_cost_us_dollars_per_kw=>40, :macrs_option_years=>5, :macrs_bonus_pct=>1, :macrs_itc_reduction=>0.5, :federal_itc_pct=>0.26, :state_ibi_pct=>0, :state_ibi_max_us_dollars=>10000000000.0, :utility_ibi_pct=>0, :utility_ibi_max_us_dollars=>10000000000.0, :federal_rebate_us_dollars_per_kw=>0, :state_rebate_us_dollars_per_kw=>0, :state_rebate_max_us_dollars=>10000000000.

["Feature Level Assumptions", [{:Scenario=>{:Site=>{:Financial=>{:om_cost_escalation_pct=>0.025, :escalation_pct=>0.023, :offtaker_tax_pct=>0.26, :offtaker_discount_pct=>0.083, :analysis_years=>25, :value_of_lost_load_us_dollars_per_kwh=>100, :microgrid_upgrade_cost_pct=>0.3}, :LoadProfile=>{:year=>2018, :loads_kw_is_net=>true, :critical_loads_kw_is_net=>false, :critical_load_pct=>0.5, :outage_is_major_event=>true}, :ElectricTariff=>{:add_blended_rates_to_urdb_rate=>false, :net_metering_limit_kw=>0, :interconnection_limit_kw=>100000000.0, :urdb_label=>"5e162e2a5457a3d50873e3af"}, :Wind=>{:min_kw=>0, :max_kw=>0, :installed_cost_us_dollars_per_kw=>3013, :om_cost_us_dollars_per_kw=>40, :macrs_option_years=>5, :macrs_bonus_pct=>1, :macrs_itc_reduction=>0.5, :federal_itc_pct=>0.26, :state_ibi_pct=>0, :state_ibi_max_us_dollars=>10000000000.0, :utility_ibi_pct=>0, :utility_ibi_max_us_dollars=>10000000000.0, :federal_rebate_us_dollars_per_kw=>0, :state_rebate_us_dollars_per_kw=>0, :state_rebat

In [30]:
p 'Scenario REopt Response',reopt_post_processor.scenario_reopt_default_output_file
p 'New Scenario Timeseries CSV', reopt_post_processor.scenario_timeseries_default_output_file
p 'Feature REopt Responses',reopt_post_processor.feature_reports_reopt_default_output_files
p 'New Feature Timeseries CSVs', reopt_post_processor.feature_reports_timeseries_default_output_files

"Scenario REopt Response"
"./run/baseline_scenario/scenario_report_Baseline Scenario_reopt_run.json"
"New Scenario Timeseries CSV"
"./run/baseline_scenario/scenario_report_Baseline Scenario_timeseries.csv"
"Feature REopt Responses"
["/Users/tkwasnik/github/urbanopt-example-reopt-project/run/baseline_scenario/1/feature_report_1_reopt_run.json", "/Users/tkwasnik/github/urbanopt-example-reopt-project/run/baseline_scenario/2/feature_report_2_reopt_run.json", "/Users/tkwasnik/github/urbanopt-example-reopt-project/run/baseline_scenario/3/feature_report_3_reopt_run.json"]
"New Feature Timeseries CSVs"
["/Users/tkwasnik/github/urbanopt-example-reopt-project/run/baseline_scenario/1/feature_report_1_timeseries.csv", "/Users/tkwasnik/github/urbanopt-example-reopt-project/run/baseline_scenario/2/feature_report_2_timeseries.csv", "/Users/tkwasnik/github/urbanopt-example-reopt-project/run/baseline_scenario/3/feature_report_3_timeseries.csv"]


["New Feature Timeseries CSVs", ["/Users/tkwasnik/github/urbanopt-example-reopt-project/run/baseline_scenario/1/feature_report_1_timeseries.csv", "/Users/tkwasnik/github/urbanopt-example-reopt-project/run/baseline_scenario/2/feature_report_2_timeseries.csv", "/Users/tkwasnik/github/urbanopt-example-reopt-project/run/baseline_scenario/3/feature_report_3_timeseries.csv"]]

In [32]:
scenario_report_scenario = reopt_post_processor.run_scenario_report(scenario_report)
scenario_report_scenario.save('baseline_scenario')

true

In [33]:
reopt_post_processor.scenario_reopt_default_assumptions_hash = nil
scenario_report_scenario_default = reopt_post_processor.run_scenario_report(scenario_report,nil,
  "./run/baseline_scenario/scenario_report_Baseline Scenario_reopt_run_DEFAULT.json",
  "./run/baseline_scenario/scenario_report_Baseline Scenario_timeseries_DEFAULT.csv")
scenario_report_scenario_default.save('baseline_scenario_DEFAULT')

true

In [34]:
scenario_report_features = reopt_post_processor.run_scenario_report_features(scenario_report)
scenario_report_features.save('baseline_features')

true

## Results

In [19]:
s_pv = scenario_report_scenario.distributed_generation.solar_pv[0].size_kw
s_storage_kw = scenario_report_scenario.distributed_generation.storage[0] .size_kw
s_storage_kwh = scenario_report_scenario.distributed_generation.storage[0].size_kwh
s_lcc = scenario_report_scenario.distributed_generation.lcc_us_dollars
s_savings = scenario_report_scenario.distributed_generation.npv_us_dollars/scenario_report_scenario.distributed_generation.lcc_us_dollars

f_pv = scenario_report_features.distributed_generation.solar_pv[0].size_kw
f_storage_kw = scenario_report_features.distributed_generation.storage[0].size_kw
f_storage_kwh = scenario_report_features.distributed_generation.storage[0].size_kwh
f_lcc = scenario_report_features.distributed_generation.lcc_us_dollars
f_pct_savings = scenario_report_features.distributed_generation.npv_us_dollars/scenario_report_features.distributed_generation.lcc_us_dollars

true

true

In [20]:
#PV Capacity
df = Daru::DataFrame.new({level:['scenario','feature'], solar_pv_kw:[s_pv,f_pv]})
p1 = df.plot type: :bar, x: :level, y: :solar_pv_kw do |plot, diagram|
  plot.x_label ""
  plot.y_label "kW"
end

#<Nyaplot::Plot:0x007fd306c13b90 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007fd306c12268 @properties={:type=>:bar, :options=>{:x=>:level, :y=>:solar_pv_kw}, :data=>"fbc36b5c-4835-45fa-ac12-b4652a15acdb"}, @xrange=["scenario", "feature"], @yrange=[0, 0.0]>], :options=>{:x_label=>"", :y_label=>"kW", :width=>700, :xrange=>["scenario", "feature"], :yrange=>[0, 0.0]}}>

In [37]:
#Storage kW
df = Daru::DataFrame.new({level:['scenario','feature'], solar_pv_kw:[s_storage_kw,f_storage_kw]})
p2 = df.plot type: :bar, x: :level, y: :solar_pv_kw do |plot, diagram|
  plot.x_label ""
  plot.y_label "kW"
end

#<Nyaplot::Plot:0x007fc4d74c1208 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007fc4d73f3088 @properties={:type=>:bar, :options=>{:x=>:level, :y=>:solar_pv_kw}, :data=>"969ad864-f890-4dc2-a38e-b95c5a3f3cc3"}, @xrange=["scenario", "feature"], @yrange=[0, 772.6976000000001]>], :options=>{:x_label=>"", :y_label=>"kW", :width=>700, :xrange=>["scenario", "feature"], :yrange=>[0, 772.6976000000001]}}>

In [38]:
#Storage kWh
df = Daru::DataFrame.new({level:['scenario','feature'], solar_pv_kw:[s_storage_kwh,f_storage_kwh]})
p3 = df.plot type: :bar, x: :level, y: :solar_pv_kw do |plot, diagram|
  plot.x_label ""
  plot.y_label "kWh"
end

#<Nyaplot::Plot:0x007fc4d67c33d0 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007fc4d67c11e8 @properties={:type=>:bar, :options=>{:x=>:level, :y=>:solar_pv_kw}, :data=>"749a6dc0-fd2d-441c-a153-dbe35697c3b6"}, @xrange=["scenario", "feature"], @yrange=[0, 2033.0894]>], :options=>{:x_label=>"", :y_label=>"kWh", :width=>700, :xrange=>["scenario", "feature"], :yrange=>[0, 2033.0894]}}>

In [39]:
#Lifecycle Costs
df = Daru::DataFrame.new({level:['scenario','feature'], solar_pv_kw:[s_lcc,f_lcc]})
p4 = df.plot type: :bar, x: :level, y: :solar_pv_kw do |plot, diagram|
  plot.x_label ""
  plot.y_label "$"
end

#<Nyaplot::Plot:0x007fc4d6199298 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007fc4d61032e8 @properties={:type=>:bar, :options=>{:x=>:level, :y=>:solar_pv_kw}, :data=>"c499ce8b-cdc1-49aa-9a11-4a877c01bc19"}, @xrange=["scenario", "feature"], @yrange=[0, 38103224.0]>], :options=>{:x_label=>"", :y_label=>"$", :width=>700, :xrange=>["scenario", "feature"], :yrange=>[0, 38103224.0]}}>

In [40]:
#Percent Savings
df = Daru::DataFrame.new({level:['scenario','feature'], solar_pv_kw:[s_savings,f_pct_savings]})
p5 = df.plot type: :bar, x: :level, y: :solar_pv_kw do |plot, diagram|
  plot.x_label ""
  plot.y_label "%"
end

#<Nyaplot::Plot:0x007fc4d580b410 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007fc4d58094d0 @properties={:type=>:bar, :options=>{:x=>:level, :y=>:solar_pv_kw}, :data=>"f7e2b625-ed0e-4718-a969-3ed424dca9b8"}, @xrange=["scenario", "feature"], @yrange=[0, 0.06527850719713615]>], :options=>{:x_label=>"", :y_label=>"%", :width=>700, :xrange=>["scenario", "feature"], :yrange=>[0, 0.06527850719713615]}}>